# **Evaluate LLMs Using LLMs**

### Overview
In this demonstration, you will evaluate the output of a simple chat LLM using an LLM. In this case, you will use GPT35-turbo-instruct to evaluate chat outputs for the following metrics:
1. Fluency = 
2. Coherence = 
3. Relevance = 

After utilizing Azure PromptFlow to generate and evaluate chat responses, this notebook will take a deeper look at the results.

### Go Deeper
The inspiration for this demonstration comes from the paper [Are Large Language Model-based Evaluators the Solution to Scaling Up Multilingual Evaluation?](https://ar5iv.labs.arxiv.org/html/2309.07462).

### Prerequisites
Ensure that your environment is setup by completing the steps outlines in [0_setup.ipynb](./0_setup.ipynb)

## 1. Upload Sample Input Data

## 2. Run Simple Chat Prompt Flow

## 3. Run Evaluation Flow

## 4.  Analyze Outputs